# Speed testing

Purpose of this Notebook is to test the speed of different packages/functions
to achieve a fast simulation process

---


### Networkit vs Netowrkx vs Graph Tool

The purpose is to test which methods from networkx or networkit or graph tool are faster 
to ensure optimal coding experience

In [1]:
"""Import necessary packages."""
import networkit as nk
import networkx as nx
import numpy as np

---

Random Graph Speed Test

In [2]:
"""Testing random graph speed."""
n_nodes = 1000
prob = 0.3

print("Networkx:")
%timeit nx.erdos_renyi_graph(n=n_nodes, p=prob)
nk.overview(nk.nxadapter.nx2nk(nx.erdos_renyi_graph(n=n_nodes, p=prob)))

print("\nNetworkit:")
%timeit nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()
nk.overview(nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate())

Networkx:
362 ms ± 4.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Network Properties:
nodes, edges			1000, 150046
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.300392
clustering coefficient		0.300423
min/max/avg degree		255, 350, 300.092000
degree assortativity		-0.000832
number of connected components	1
size of largest component	1000 (100.00 %)

Networkit:
19.4 ms ± 6.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Network Properties:
nodes, edges			1000, 149683
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.299666
clustering coefficient		0.299697
min/max/avg degree		251, 344, 299.366000
degree assortativity		-0.001637
number of connected components	1
size of largest component	1000 (100.00 %)


---

Barabasi-Albert Speed Test


In [3]:
"""Testing Barabasi-speed test."""
n_nodes = 1000
n_edges = 3

print("Networkx:")
%timeit nx.barabasi_albert_graph(n=n_nodes, m=n_edges)
nk.overview(nk.nxadapter.nx2nk(nx.barabasi_albert_graph(n=n_nodes, m=n_edges)))

print("\nNetworkit:")
%timeit nk.generators.BarabasiAlbertGenerator(k=n_edges,nMax=n_nodes).generate()
nk.overview(nk.generators.BarabasiAlbertGenerator(k=n_edges, nMax=n_nodes).generate())

Networkx:
33.8 ms ± 18.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Network Properties:
nodes, edges			1000, 2991
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.005988
clustering coefficient		0.027871
min/max/avg degree		3, 81, 5.982000
degree assortativity		0.281362
number of connected components	1
size of largest component	1000 (100.00 %)

Networkit:
1.19 ms ± 461 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Network Properties:
nodes, edges			1000, 2994
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.005994
clustering coefficient		0.033799
min/max/avg degree		3, 106, 5.988000
degree assortativity		0.231879
number of connected components	1
size of largest component	1000 (100.00 %)



---
Coversion Speed Test

In [4]:
"""Testing conversion speed."""
n_nodes = 1000
prob = 0.3

print("Networkx:")
graph = nx.erdos_renyi_graph(n=n_nodes, p=prob)
%timeit nk.nxadapter.nx2nk(graph)

print("Networkit:")
graph = nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()
%timeit nk.nxadapter.nk2nx(graph)

Networkx:
95.8 ms ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Networkit:
303 ms ± 68.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

### Testing the speed of various functions

The purpose is to optimise python in order to have the best results once dealing with intense networks

Testing if yield is faster than return with the fibonacci sequence

In [7]:
def fib_yield(n):
    """Fibonacci sequence with yield."""
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b


def fib_return(n):
    """Fibonacci sequence with return."""
    a, b = 0, 1
    sequence = []
    for _ in range(n):
        sequence.append(a)
        a, b = b, a + b


n = 100000
print("Fibonacci sequence with yield:")
%timeit list(fib_yield(n))

print("\nFibonacci sequence with return:")
%timeit fib_return(n)

Fibonacci sequence with yield:
638 ms ± 235 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Fibonacci sequence with return:
538 ms ± 83.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
Testing if IF condition is faster than collect the all the values and get the min value

In [11]:
n_edges = 4
n_nodes = 50
random_network = nk.generators.BarabasiAlbertGenerator(
    k=n_edges, nMax=n_nodes
).generate()


def get_radius_if_cond(network):
    """Get the radius (min eccentricity) using if condition."""
    radius = np.inf

    for node in network.iterNodes():
        new_radius = nk.distance.Eccentricity.getValue(network, node)[1]
        if new_radius < radius:
            radius = new_radius

    return radius


def get_radius_def_list(network):
    """Get the radius (min eccentricity) using if condition."""
    # predefine the len of the list for speed
    radius = np.zeros(random_network.numberOfNodes())
    # to append to the right idx in the list
    iterator = iter(range(0, network.numberOfNodes()))

    for node in network.iterNodes():
        radius[next(iterator)] = nk.distance.Eccentricity.getValue(network, node)[1]

    return min(radius)


print("Radius with if condition:")
%timeit get_radius_if_cond(random_network)
print(f"Radius is {get_radius_if_cond(random_network)}")

print("\nRadius with predefined list:")
%timeit get_radius_def_list(random_network)
print(f"Radius is {get_radius_def_list(random_network)}")

Radius with if condition:
34 ms ± 4.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Radius is 4

Radius with predefined list:
29 ms ± 384 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Radius is 4.0

Radius with multiprocessing:


NameError: name 'network' is not defined